In [1]:
import os 

import acquire

import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer

from env import get_connection

from scipy.stats import mannwhitneyu

from scipy import stats

from env import get_connection

from sklearn.metrics import accuracy_score

from sklearn.metrics import precision_score

from sklearn.metrics import recall_score

from sklearn.metrics import confusion_matrix

## Exercises


## What is your baseline prediction?
## What is your baseline accuracy? 
## remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [2]:
df = acquire.get_titanic_data()

found data


In [3]:
df = acquire.prep_titanic(df)

In [4]:
df.survived.value_counts()

0    549
1    342
Name: survived, dtype: int64

In [5]:
train, val, test = acquire.train_val_test(df, 'survived')

In [6]:
train.shape, val.shape

((623, 10), (134, 10))

In [7]:
train.head()

,survived,sex,age,sibsp,parch,fare,class,deck,embark_town,alone
748,0,male,19.0,1,0,53.1000,First,D,Southampton,0
45,0,male,NaN,0,0,8.0500,Third,NaN,Southampton,1
28,1,female,NaN,0,0,7.8792,Third,NaN,Queenstown,1
633,0,male,NaN,0,0,0.0000,First,NaN,Southampton,1
403,0,male,28.0,1,0,15.8500,Third,NaN,Southampton,0


In [8]:
val.head()

,survived,sex,age,sibsp,parch,fare,class,deck,embark_town,alone
377,0,male,27.0,0,2,211.500,First,C,Cherbourg,0
244,0,male,30.0,0,0,7.225,Third,NaN,Cherbourg,1
72,0,male,21.0,0,0,73.500,Second,NaN,Southampton,1
815,0,male,NaN,0,0,0.000,First,B,Southampton,1
841,0,male,16.0,0,0,10.500,Second,NaN,Southampton,1


In [9]:
train['baseline'] = 0
train.head()

,survived,sex,age,sibsp,parch,fare,class,deck,embark_town,alone,baseline
748,0,male,19.0,1,0,53.1000,First,D,Southampton,0,0
45,0,male,NaN,0,0,8.0500,Third,NaN,Southampton,1,0
28,1,female,NaN,0,0,7.8792,Third,NaN,Queenstown,1,0
633,0,male,NaN,0,0,0.0000,First,NaN,Southampton,1,0
403,0,male,28.0,1,0,15.8500,Third,NaN,Southampton,0,0


In [10]:
pd.crosstab(train.survived, train.baseline)

baseline,0
survived,
0,384
1,239


In [11]:
base_acc = accuracy_score(train.survived, train.baseline)
base_acc

0.6163723916532905

In [12]:
(train.survived == 0).mean()

0.6163723916532905

In [13]:
(train.baseline == train.survived).mean()

0.6163723916532905

## Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [14]:
train = pd.get_dummies(train, columns = ['sex'], drop_first = True)
train = pd.get_dummies(train, columns = ['class', 'embark_town'])
train.head()
                                    

,survived,age,sibsp,parch,fare,deck,alone,baseline,sex_male,class_First,class_Second,class_Third,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
748,0,19.0,1,0,53.1000,D,0,0,1,1,0,0,0,0,1
45,0,NaN,0,0,8.0500,NaN,1,0,1,0,0,1,0,0,1
28,1,NaN,0,0,7.8792,NaN,1,0,0,0,0,1,0,1,0
633,0,NaN,0,0,0.0000,NaN,1,0,1,1,0,0,0,0,1
403,0,28.0,1,0,15.8500,NaN,0,0,1,0,0,1,0,0,1


In [15]:
train = train.drop(columns = ['age', 'deck'])

In [16]:
X_train = train.drop(columns = ['survived'])
y_train = train.survived

In [17]:
val['baseline'] = 0
val.head()

,survived,sex,age,sibsp,parch,fare,class,deck,embark_town,alone,baseline
377,0,male,27.0,0,2,211.500,First,C,Cherbourg,0,0
244,0,male,30.0,0,0,7.225,Third,NaN,Cherbourg,1,0
72,0,male,21.0,0,0,73.500,Second,NaN,Southampton,1,0
815,0,male,NaN,0,0,0.000,First,B,Southampton,1,0
841,0,male,16.0,0,0,10.500,Second,NaN,Southampton,1,0


In [18]:
val = pd.get_dummies(val, columns = ['sex'], drop_first = True)
val = pd.get_dummies(val, columns = ['class', 'embark_town'])
val.head()

,survived,age,sibsp,parch,fare,deck,alone,baseline,sex_male,class_First,class_Second,class_Third,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
377,0,27.0,0,2,211.500,C,0,0,1,1,0,0,1,0,0
244,0,30.0,0,0,7.225,NaN,1,0,1,0,0,1,1,0,0
72,0,21.0,0,0,73.500,NaN,1,0,1,0,1,0,0,0,1
815,0,NaN,0,0,0.000,B,1,0,1,1,0,0,0,0,1
841,0,16.0,0,0,10.500,NaN,1,0,1,0,1,0,0,0,1


In [19]:
val = val.drop(columns = ['age', 'deck'])

In [20]:
X_val = val.drop(columns = ['survived'])
y_val = val.survived

In [21]:
seed = 42

clf = DecisionTreeClassifier(max_depth = 20, random_state = seed)

clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=20, random_state=42)

## Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [22]:
train_preds = clf.predict(X_train)
train_preds[:5]

array([0, 0, 1, 0, 0])

In [23]:
clf.score(X_train, y_train)

0.9309791332263242

In [24]:
co = confusion_matrix(y_train, train_preds)
co

array([[377,   7],
       [ 36, 203]])

In [25]:
pd.crosstab(y_train, train_preds)

col_0,0,1
survived,,
0,377,7
1,36,203


In [26]:
y_train.value_counts()

0    384
1    239
Name: survived, dtype: int64

In [27]:
labels = sorted(y_train.unique())

pd.DataFrame(co, index = labels, columns = labels)

,0,1
0,377,7
1,36,203


In [28]:
print(classification_report(y_train, train_preds))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95       384
           1       0.97      0.85      0.90       239

    accuracy                           0.93       623
   macro avg       0.94      0.92      0.93       623
weighted avg       0.93      0.93      0.93       623



In [29]:
clf.score(X_val, y_val)

0.7835820895522388

In [30]:
val_preds = clf.predict(X_val)
val_preds[:5]

array([1, 0, 0, 0, 0])

In [31]:
print(classification_report(y_val, val_preds))

              precision    recall  f1-score   support

           0       0.82      0.83      0.82        82
           1       0.73      0.71      0.72        52

    accuracy                           0.78       134
   macro avg       0.77      0.77      0.77       134
weighted avg       0.78      0.78      0.78       134



## Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [32]:
pd.crosstab(y_train, train_preds)

col_0,0,1
survived,,
0,377,7
1,36,203


In [33]:
accuracy = (train_preds == y_train).mean()
accuracy

0.9309791332263242

In [34]:
subset = train[y_train == 0]
subset.head()

,survived,sibsp,parch,fare,alone,baseline,sex_male,class_First,class_Second,class_Third,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
748,0,1,0,53.10,0,0,1,1,0,0,0,0,1
45,0,0,0,8.05,1,0,1,0,0,1,0,0,1
633,0,0,0,0.00,1,0,1,1,0,0,0,0,1
403,0,1,0,15.85,0,0,1,0,0,1,0,0,1
695,0,0,0,13.50,1,0,1,0,1,0,0,0,1


In [35]:
recall = 203 / (203 + 36)
recall

0.8493723849372385

In [36]:
recall = (subset.survived == train_preds).mean()
recall

ValueError: ('Lengths must match to compare', (384,), (623,))

In [ ]:
precision = 203 / (203 + 7)
precision

## Run through steps 2-4 using a different max_depth value.

In [ ]:
seed = 42

clf = DecisionTreeClassifier(max_depth = 10, random_state = seed)

clf.fit(X_train, y_train)

## Which model performs better on your in-sample data?

## Which model performs best on your out-of-sample data, the validate set?

## Work through these same exercises using the Telco dataset.

In [37]:
df = acquire.get_telco_data()
df.head()

found data


,contract_type_id,contract_type,customer_id,contract_type_id.1,paperless_billing,customer_id.1,churn_month,customer_id.2,gender,senior_citizen,...,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,internet_service_type_id.1,internet_service_type,payment_type_id.1,payment_type
0,1,Month-to-month,3308-MHOOC,1,Yes,NaN,NaN,3308-MHOOC,Male,0,...,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,3,None,2,Mailed check
1,1,Month-to-month,3307-TLCUD,1,No,NaN,NaN,3307-TLCUD,Male,0,...,Yes,No,Yes,No,No,No,1,DSL,2,Mailed check
2,1,Month-to-month,3301-LSLWQ,1,Yes,NaN,NaN,3301-LSLWQ,Female,0,...,Yes,No,No,No,No,Yes,1,DSL,1,Electronic check
3,1,Month-to-month,3296-SILRA,1,Yes,3296-SILRA,2022-01-31,3296-SILRA,Female,1,...,No,Yes,No,No,No,No,2,Fiber optic,2,Mailed check
4,1,Month-to-month,3295-YVUSR,1,Yes,NaN,NaN,3295-YVUSR,Male,1,...,No,No,No,No,No,No,2,Fiber optic,2,Mailed check


In [38]:
def prep_telco(df):
    
    df = df.drop(columns = ['contract_type_id', 'contract_type_id.1', 'customer_id.1', 'customer_id.2', 'customer_id.3', 'customer_id.4', 'customer_id.5', 'internet_service_type_id', 'internet_service_type_id.1', 'contract_type_id', 'contract_type_id.1', 'payment_type_id', 'payment_type_id.1'])
    
    return df

In [39]:
df = prep_telco(df)
df.head()

,contract_type,customer_id,paperless_billing,churn_month,gender,senior_citizen,partner,dependents,monthly_charges,total_charges,...,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,internet_service_type,payment_type
0,Month-to-month,3308-MHOOC,Yes,NaN,Male,0,No,Yes,19.2,19.2,...,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,None,Mailed check
1,Month-to-month,3307-TLCUD,No,NaN,Male,0,Yes,No,34.4,592.75,...,No,No phone service,Yes,No,Yes,No,No,No,DSL,Mailed check
2,Month-to-month,3301-LSLWQ,Yes,NaN,Female,0,No,No,66.1,1912.15,...,Yes,Yes,Yes,No,No,No,No,Yes,DSL,Electronic check
3,Month-to-month,3296-SILRA,Yes,2022-01-31,Female,1,Yes,No,76.4,76.4,...,Yes,No,No,Yes,No,No,No,No,Fiber optic,Mailed check
4,Month-to-month,3295-YVUSR,Yes,NaN,Male,1,No,No,72.6,154.3,...,Yes,Yes,No,No,No,No,No,No,Fiber optic,Mailed check


## Experiment with this model on other datasets with a higher number of output classes.